# Reconstructing out-of-sample DVs

Given a quantitative ontology, or psychological space, that DVs can be projected into, how can we deterine the embedding of new variables?

Currently, our embedding is determined by factor analysis. Thus ontological embedding are only known for the DVs entered into the original model. How could we extend this?

One possibility is measuring new variables in the same population that completed our original battery. After doing this we could either (1) run the model anew, or (2) use linear regression to map the already discovered factors onto the new variables. The former is better, but results in small changes to the actual factors with each new variable. The latter method ensures that our factors stay the same. Neither is scalable, however, as we do not, in general, have access to a constant population that can be remeasured whenever new measures come into the picture.

Another possibility that works with new populations requires that the new population completes the entire battery used to estimate the original factors, in addition to whatever new variables are of interest. Doing so allows the calculation of factor scores for this new population based on the original model, which can then be mapped to the new measures of interest. This allows researchers to capitalize on the original model (presumably fit on more subjects than the new study), while expanding the ontology. Problems exist here, however.
- The most obvious problem is that you have to measure the new sample on the entire battery used to fit the original EFA model. Given that this takes many hours (the exact number depending on whether tasks, surveys or both are used), this is exceedingly impractical. In our cas we did have our new fMRI sample take the entire battery (or at least a subset of participants), so this problem isn't as relevant
- Still problems remain. If N is small, the estimate of the ontological embedding for new DVs is likely unstable.

This latter problem necessitates some quantitative exploration. This notebook simulates the issue by:
1. Removing a DV from the original ontology dataset
2. Performing EFA on this subset
3. Using linear regression to map these EFA factors to the left out variable

(3) is performed on smaller population sizes to reflect the reality of most studies (including ours) and is repeated to get a sense of the mapping's variability

### Small issues not currently addressed

- The EFA model is fit on the entire population. An even more stringent simulation would subset the subjects used in the "new study" and fit the EFA model on a completely independent group. I tried this once - the factor scores hardly differed. In addition, I want the EFA model to be as well-powered as possible, as that will be the reality for this method moving forward
- I am currently not holding out entire tasks, but only specific DVs

In [ ]:
import argparse
from glob import glob
import numpy as np
from os import makedirs, path
import pandas as pd
import pickle
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.preprocessing import normalize

from dimensional_structure.EFA_plots import get_communality
from dimensional_structure.utils import abs_pdist
from ontology_mapping.reconstruction_plots import (plot_factor_reconstructions,
                                                       plot_reconstruction_hist)
from ontology_mapping.reconstruction_utils import (get_reconstruction_results, 
                                                   linear_reconstruction,
                                                   k_nearest_reconstruction,
                                                  CV_predict)
from selfregulation.utils.plot_utils import format_num, save_figure
from selfregulation.utils.result_utils import load_results
from selfregulation.utils.utils import get_recent_dataset, get_info, get_retest_data

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
import sklearn
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
%matplotlib inline
# import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import pdist, squareform

In [ ]:
# ignore some warnings
import warnings
warnings.filterwarnings("ignore", category=sklearn.metrics.classification.UndefinedMetricWarning)

In [ ]:
# argparse
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-pop_sizes', nargs='+', default=[25, 50, 100, 400], type=int)
    parser.add_argument('-n_reps', default=200)
    parser.add_argument('-n_measures', default=None, type=int)
    parser.add_argument('-dataset', default=None)
    parser.add_argument('-rerun', action='store_true')
    parser.add_argument('-append', action='store_true')
    parser.add_argument('-EFA_rotation', default='oblimin')
    parser.add_argument('-knn_metric', default='correlation')
    parser.add_argument('-verbose', action='store_true')
    parser.add_argument('-save', action='store_true')
    args, _ = parser.parse_known_args()
    pop_sizes = args.pop_sizes
    n_reps = args.n_reps
    n_measures = args.n_measures
    rerun = args.rerun
    append = args.append
    knn_metric = args.knn_metric
    EFA_rotation = args.EFA_rotation
    verbose = args.verbose
    save = args.save
    if args.dataset is not None:
        dataset = args.dataset
    else:
        dataset = get_recent_dataset()

In [ ]:
# additional setup
np.random.seed(12412)
results = load_results(dataset)['task']
retest_data = get_retest_data(dataset.replace('Complete', 'Retest'))
c = results.EFA.results['num_factors']

classifiers = {'Ridge': Ridge(fit_intercept=False),
               'LR': LinearRegression(fit_intercept=False)}
# get output dir to store results
output_dir = path.join(get_info('results_directory'),
                       'ontology_reconstruction', results.ID, EFA_rotation)
makedirs(output_dir, exist_ok=True)
# get plot dir to store plots
plot_dir = path.join(output_dir, 'Plots')
makedirs(plot_dir, exist_ok=True)

In [ ]:
# get a random subset of variables to perform the calculation on if n_vars is set
measures = np.unique([i.split('.')[0] for i in results.data.columns])
if n_measures is not None:
    measure_list = np.random.choice(measures, n_measures, replace=False)
else:
    measure_list = measures
# get all variables from selected tasks
var_list = results.data.filter(regex='|'.join(measure_list)).columns

In [ ]:
def load_files(reconstruction_files, query_string=None):
    out = {}
    for f in reconstruction_files:
        tmp = pd.read_pickle(f)
        if query_string:
            tmp = tmp.query(query_string)
        name = f.split('-')[-1][:-4]
        out[name] = tmp
    return out

def update_files(old, new):
    for k, df in old.items():
        add = new.pop(k)
        add = add.query('label == "partial_reconstruct"')
        add.loc[:,'rep'] += df.rep.max()
        old[k] = pd.concat([df, add], sort=False).reset_index(drop=True)
    old.update(new)
        
def combine_files(reconstruction_files, query_string=None):
    if type(reconstruction_files) != dict:
        out = load_files(reconstruction_files, query_string)
    else:
        out = reconstruction_files
    return pd.concat(out, sort=False).reset_index(drop=True)

def normalize_reconstruction(reconstruction, c, inplace=True):
    """ Ensures reconstructions lie on the unit circle """
    if not inplace:
        reconstruction = reconstruction.copy()
    normed = normalize(reconstruction.iloc[:,:c])
    reconstruction.iloc[:,:c] = normed
    if not inplace:
        return reconstruction
    

Run simulation for every variable at different population sizes. 

That is, do the following:

1. take a variable (say stroop incongruent-congruent RT), remove it from the data matrix
2. Run EFA on the data matrix composes of the 522 (subject) x N-1 (variable) data matrix
3. Calculate factor scores for all 522 subjects
4. Select a subset of "pop_size" to do an "ontological mapping". That is, pretend that these subjects did the whole battery (missing the one variable) *and then* completed one more task. The idea is we want to do a mapping from those subject's factor scores to the new variable
   1. We can do a linear mapping (regression) from the ontological scores to the output variable
   2. We can do a k-nearest neighbor interpolation, where we say the unknown ontological factor is a blend of the "nearest" variables in the dataset
5. Repeat (4) a number of times to get a sense for the accuracy and variability of that mapping
6. Compare the estimated ontological scores for the held out var (stroop incongruent-congruent) to the original "correct" ontological mapping (that would have been obtained if the variable was included in the original data matrix

## Perform reconstruction

### K Nearest Neighbor Reconstruction

In [ ]:
%%time
k_list = list(range(1,20))
basename = path.join(output_dir, 'KNNR_%s-*' % knn_metric)
files = glob(basename)
updated = []
if rerun: # rerun everything
    regex_list = ['^'+m for m in measure_list]
    k_reconstructions=get_reconstruction_results(results, regex_list, pop_sizes, 
                                                 n_reps=n_reps, 
                                                 recon_fun=k_nearest_reconstruction, 
                                                 k_list=k_list, 
                                                 metric=knn_metric,
                                                 EFA_rotation=EFA_rotation,
                                                 verbose=verbose)
    updated = list(k_reconstructions)
else:
    k_reconstructions = load_files(files)
    if append: # add more simulations to previous files
        regex_list = ['^'+m for m in measure_list]
        to_append = get_reconstruction_results(results, regex_list, pop_sizes, 
                                                n_reps=n_reps, 
                                                recon_fun=k_nearest_reconstruction, 
                                                k_list=k_list, 
                                                metric=knn_metric,
                                                EFA_rotation=EFA_rotation,
                                               verbose=verbose)
        updated = list(to_append.keys())
        update_files(k_reconstructions, to_append)
    else: # load previous files and add run any additional ones required
        tmp_measures = set(measure_list) - set(k_reconstructions.keys())
        regex_list = ['^'+m for m in tmp_measures]
        additional = get_reconstruction_results(results, regex_list, pop_sizes, 
                                                n_reps=n_reps, 
                                                recon_fun=k_nearest_reconstruction, 
                                                k_list=k_list, 
                                                metric=knn_metric,
                                                EFA_rotation=EFA_rotation,
                                                verbose=verbose)
        k_reconstructions.update(additional)
        updated = additional.keys()

for measure in updated:
    df = k_reconstructions[measure]
    if save:
        df.to_pickle(basename[:-1]+'%s.pkl' % measure)

In [ ]:
%%time
k_list = list(range(1,20))
basename = path.join(output_dir, 'KNNRind_%s-*' % knn_metric)
files = glob(basename)
updated = []
pop_size_subset = [i for i in pop_sizes if i < 100]
if rerun: # rerun everything
    regex_list = ['^'+m for m in measure_list]
    k_reconstructions_ind=get_reconstruction_results(results, regex_list, pop_size_subset, 
                                                 n_reps=n_reps, 
                                                 recon_fun=k_nearest_reconstruction, 
                                                 k_list=k_list, 
                                                 metric=knn_metric,
                                                 EFA_rotation=EFA_rotation,
                                                 independent_EFA=True,
                                                 verbose=verbose)
    updated = measure_list
else:
    k_reconstructions_ind = load_files(files)
    if append: # add more simulations to previous files
        regex_list = ['^'+m for m in measure_list]
        to_append = get_reconstruction_results(results, regex_list, pop_size_subset, 
                                                n_reps=n_reps, 
                                                recon_fun=k_nearest_reconstruction, 
                                                k_list=k_list, 
                                                metric=knn_metric,
                                                EFA_rotation=EFA_rotation,
                                                independent_EFA=True,
                                                verbose=verbose)
        updated = list(to_append.keys())
        update_files(k_reconstructions_ind, to_append)
    else: # load previous files and add run any additional ones required
        tmp_measures = set(measure_list) - set(k_reconstructions_ind.keys())
        regex_list = ['^'+m for m in tmp_measures]
        additional = get_reconstruction_results(results, regex_list, pop_size_subset, 
                                                n_reps=n_reps, 
                                                recon_fun=k_nearest_reconstruction, 
                                                k_list=k_list, 
                                                metric=knn_metric,
                                                EFA_rotation=EFA_rotation,
                                                independent_EFA=True,
                                                verbose=verbose)
        k_reconstructions_ind.update(additional)
        updated = additional.keys()

for measure in updated:
    df = k_reconstructions_ind[measure]
    if save:
        df.to_pickle(basename[:-1]+'-%s.pkl' % measure)

In [ ]:
def summarize_k(k_reconstructions):
    var_summary = pd.DataFrame()
    for measure, reconstruction in k_reconstructions.items():
        tmp_summary = reconstruction.query('label=="partial_reconstruct"') \
                        .groupby(['pop_size', 'k', 'weighting','var'])['corr_score'].mean().reset_index()
        var_summary = pd.concat([var_summary, tmp_summary])
    
    k_summary = var_summary.groupby(['pop_size', 'k', 'weighting']).mean()
    # summarize further
    k_best_params = {}
    for pop_size in k_summary.index.unique(level='pop_size'):
        tmp=k_summary.query('pop_size == %s' % pop_size)
        best_params = tmp.idxmax()[0]
        best_val = tmp.loc[best_params][0]
        k_best_params[pop_size] = {'k': best_params[1], 
                                   'weighting': best_params[2],
                                   'best_val': best_val}
    reconstruction_list = []
    for reconstruction in k_reconstructions.values():
        true = reconstruction.query('label == "true"')
        reconstruction_list.append(true)
        for k, v in k_best_params.items():
            tmp_partial = reconstruction.query('pop_size == %s and \
                                         k == %s and \
                                         weighting == "%s"' % (k, v['k'], v['weighting']))
            full = reconstruction.query('label == "full_reconstruct" and \
                                         k == %s and \
                                         weighting == "%s"' % (v['k'], v['weighting']))
            reconstruction_list += [tmp_partial, full]

    k_best_reconstruction = pd.concat(reconstruction_list, axis=0, sort=False)
    return var_summary, k_best_params, k_best_reconstruction

In [ ]:
k_var_summary, k_best_params, k_reconstruction = summarize_k(k_reconstructions)
k_reconstruction.query('label=="partial_reconstruct"') \
    .groupby('pop_size')['corr_score'].agg(['mean','std'])

In [ ]:
k_ind_var_summary, k_best_params, k_ind_reconstruction = summarize_k(k_reconstructions_ind)
k_ind_reconstruction.query('label=="partial_reconstruct"') \
    .groupby('pop_size')['corr_score'].agg(['mean','std'])

In [ ]:
del k_reconstructions

### Linear Reconstruction

In [ ]:
%%time
clfs = {'Linear': LinearRegression(fit_intercept=False),
       'RidgeCV': RidgeCV(fit_intercept=False, cv=10)}
linear_reconstructions = {}
for clf_name, clf in clfs.items():
    basename = path.join(output_dir, 'linear-%s_reconstruct*' % clf_name)
    files = glob(basename)
    updated = []
    if rerun: # rerun everything
        regex_list = ['^'+m for m in measure_list]
        tmp_reconstructions=get_reconstruction_results(results, regex_list, pop_sizes, 
                                                       n_reps=n_reps, 
                                                       recon_fun=linear_reconstruction, 
                                                       clf=clf,
                                                       EFA_rotation=EFA_rotation,
                                                       verbose=verbose)
        updated = measure_list
    else:
        tmp_reconstructions = load_files(files)
        if append: # add more simulations to previous files
            regex_list = ['^'+m for m in measure_list]
            to_append = get_reconstruction_results(results, regex_list, pop_sizes, 
                                                       n_reps=n_reps, 
                                                       recon_fun=linear_reconstruction, 
                                                       clf=clf,
                                                       EFA_rotation=EFA_rotation,
                                                       verbose=verbose)
            updated = list(to_append.keys())
            update_files(tmp_reconstructions, to_append)
        else: # load previous files and add run any additional ones required
            tmp_measures = set(measure_list) - set(tmp_reconstructions.keys())
            regex_list = ['^'+m for m in tmp_measures]
            additional = get_reconstruction_results(results, regex_list, pop_sizes, 
                                                    n_reps=n_reps, 
                                                    recon_fun=linear_reconstruction, 
                                                    clf=clf,
                                                    EFA_rotation=EFA_rotation,
                                                    verbose=verbose)
            tmp_reconstructions.update(additional)
            updated = additional.keys()

    for measure in updated:
        df = tmp_reconstructions[measure]
        if save:
            df.to_pickle(basename[:-1]+'-%s.pkl' % measure)
        
    files = glob(basename)
    if len(files) > 0:
        linear_reconstructions[clf_name] = combine_files(files)

In [ ]:
summary = pd.DataFrame()
for clf, df in linear_reconstructions.items():
    tmp = df.query('label=="partial_reconstruct"') \
        .groupby('pop_size').corr_score.agg([np.mean, np.std])
    tmp.loc[:,'clf'] = clf
    summary = pd.concat([summary, tmp], sort=False)
print(summary)

In [ ]:
reconstructions = {'KNN': k_reconstruction,
                   'RidgeCV': linear_reconstructions['RidgeCV']}
reconstructed_vars = sorted(k_reconstruction['var'].unique())
assert set(reconstructed_vars) == set(reconstructions['RidgeCV']['var'].unique())

## Statistics

### Variable characteristics that influence reconstruction quality

In [ ]:
# variable characteristics
reconstruction = reconstructions['KNN']
retest_index = [i.replace('.logTr','').replace('.ReflogTr','') for i in reconstructed_vars]
retest_vals = retest_data.loc[retest_index,'icc']
retest_vals.index = reconstructed_vars
communality = get_communality(results.EFA).loc[reconstructed_vars]
avg_corr  = abs(results.data.corr()).replace(1,0).mean()
avg_corr.name = "avg_correlation"

In [ ]:
# create summaries
additional = pd.concat([retest_vals, communality, avg_corr], axis=1)
reconstruction_summaries = {}
for name, reconstruction in reconstructions.items():
    s = reconstruction.query('label == "partial_reconstruct"') \
        .groupby(['var', 'pop_size']).corr_score.agg(['mean', 'std'])
    s = s.reset_index().join(additional, on='var')
    reconstruction_summaries[name] = s
all_reconstructions = pd.concat(reconstruction_summaries).reset_index()
all_reconstructions = all_reconstructions.rename({'level_0': 'approach'}, axis=1).drop('level_1', axis=1)

Does reconstruction success at one population size predict the next?

In [ ]:
tmp = []
for i,group in all_reconstructions.groupby(['approach', 'pop_size']):
    group = group.loc[:,['var','mean']].set_index('var')
    group.columns = [i]
    tmp.append(group)
approach_compare = pd.concat(tmp, axis=1)
approach_compare.columns = [i+': '+str(int(j)) for i,j in approach_compare.columns]
# correlation of reconstructions
corr= approach_compare.corr(method='spearman')
overall_correlation = np.mean(corr.values[np.tril_indices_from(corr, -1)])
print('DV reconstruction score correlates %s across approaches' % format_num(overall_correlation))

Model reconstruction success as a function of DV characteristics, approach and subpopulation size

In [ ]:
from patsy.contrasts import ContrastMatrix

def _name_levels(prefix, levels):
    return ["[%s%s]" % (prefix, level) for level in levels]

class Simple(object):
    def _simple_contrast(self, levels):
        nlevels = len(levels)
        contr = -1./nlevels * np.ones((nlevels, nlevels-1))
        contr[1:][np.diag_indices(nlevels-1)] = (nlevels-1.)/nlevels
        return contr

    def code_with_intercept(self, levels):
        contrast = np.column_stack((np.ones(len(levels)),
                                    self._simple_contrast(levels)))
        return ContrastMatrix(contrast, _name_levels("Simp.", levels))

    def code_without_intercept(self, levels):
        contrast = self._simple_contrast(levels)
        return ContrastMatrix(contrast, _name_levels("Simp.", levels[:-1]))

In [ ]:

mdf = md.fit()
mdf.summary()

In [ ]:
contrast = Simple().code_without_intercept([0,1])
print(contrast.matrix)
all_reconstructions.loc[:, 'z_mean'] = np.arctanh(all_reconstructions['mean'])
md = smf.mixedlm("z_mean ~ (pop_size + icc + avg_correlation)*C(approach, Sum)", all_reconstructions, groups=all_reconstructions["var"])
mdf = md.fit()
mdf.summary()

# other way to do it
# endog, exog = patsy.dmatrices("z_mean ~ (pop_size + icc + avg_correlation)*C(approach, Sum)", all_reconstructions, return_type='dataframe')
# md = sm.MixedLM(endog=endog, exog=exog, groups=all_reconstructions['var'])

### How well are we reconstructing distances?

In [ ]:
orig_loadings = results.EFA.get_loading(c=c).loc[reconstructed_vars]
orig_distances = pd.DataFrame(squareform(abs_pdist(orig_loadings)), index=orig_loadings.index, columns=orig_loadings.index)

reconstructed_distances = {}
for name, reconstruction in reconstructions.items():
    for pop_size in sorted(reconstruction.pop_size.dropna().unique()):
        reconstructed_loadings = reconstruction.query('pop_size == %s' % pop_size).groupby('var').mean().iloc[:,:c]
        distances = pd.DataFrame(squareform(abs_pdist(reconstructed_loadings)), 
                                index=orig_loadings.index, columns=orig_loadings.index)
        reconstructed_distances[name+'_%03d' % pop_size] = distances

### How well can we recover the variable from the embedding? How differentiable are the embeddings?

The recovery of distances seems like the better analysis, but keeping this in the notebook

#### Classify each variable/cluster based on its embedding

In [ ]:
labels = results.HCA.results['EFA5_oblimin']['distance_df']
clusters = results.HCA.results['EFA5_oblimin']['labels']
cluster_map = {l:c for l,c in zip(labels,clusters)}

In [ ]:
%%time
classification_scores = {}
for key, reconstruction in reconstructions.items():
    # save scores
    var_scores = {}
    cluster_scores = {}
    # set up input
    true = reconstruction.query('label == "true"')
    test_set = [true.iloc[:,:c].values]
    var_test_set = test_set + [true['var']]
    cluster_test_set = test_set + [true['var'].apply(lambda x: cluster_map[x])]
    clf=KNeighborsClassifier(weights='distance', n_neighbors=10)
    
    for pop_size in [i for i in reconstruction.pop_size.unique() if str(i) != 'nan']:
        reconstruction_subset = reconstruction.query('pop_size==%s and label=="partial_reconstruct"' % pop_size)
        var_scores[pop_size] = CV_predict(reconstruction_subset, reconstruction_subset['var'],
                                          clf=clf, cv=10, test_set=var_test_set)
        # classify based on clusters
        cluster_labels = reconstruction_subset['var'].apply(lambda x: cluster_map[x])
        cluster_scores[pop_size] = CV_predict(reconstruction_subset, cluster_labels,
                                      clf=clf, cv=10, test_set=cluster_test_set)
    classification_scores[key] = {'var_scores': var_scores,
                                  'cluster_scores': cluster_scores}

#### When the true values are misclassified, what are they classified as?

In [ ]:
# only calculating this for one method as I won't be using this analysis.
tmp_classification = classification_scores['RidgeCV']
tmp_reconstruction = reconstructions['RidgeCV']
var_scores = tmp_classification['var_scores'] 

le = LabelEncoder()
le.fit(reconstruction['var'])
confusion = var_scores[400]['true_confusion']
classification = {}
for i, confusion_vector in enumerate(confusion):
    true_label = le.inverse_transform([i])[0]
    classified_label = le.inverse_transform([np.where(confusion_vector)[0][0]])[0]
    classification[true_label] = classified_label
    if verbose and (true_label!=classified_label):
        print('True: %s\nClassified: %s' % (true_label, classified_label))
        print('*'*79)

#### How much gaussian noise equates the performance of this classifier?

In [ ]:
%%time
reps = 100
clf=KNeighborsClassifier(weights='distance', n_neighbors=10)
noisy_scores = {}
# hyperparameters
for noise_val in np.linspace(.05, .5, 20):
    true = tmp_reconstruction.query('label == "true"')
    labels = true['var'].tolist()*reps
    embeddings = true.iloc[:,:c].values
    noise = np.random.normal(0, (embeddings.std()*noise_val), size=[reps,c])
    simulated_noise = np.empty([0,c])
    for n in noise:
        simulated_noise = np.vstack([simulated_noise, embeddings+n])
    noisy_data = pd.DataFrame(simulated_noise, columns=true.columns[:c])
    noisy_data.loc[:,'var'] = labels
    noisy_scores[noise_val] = CV_predict(noisy_data, noisy_data['var'],
                                           clf=clf, cv=10)

## Visualization

Of concern is the average correspondence and variability between the estimated ontological fingerprint of a DV and its "ground-truth" (the original estimate when it was part of the EFA model)

One way to look at this is just the average reconstruction score (e.g., for example) and variability of reconstruction score as a function of pseudo-pop-size and model parameters

In [ ]:
colors = sns.color_palette('Set1', n_colors = len(pop_sizes), desat=.8)

### Overall Performance

In [ ]:
f = plt.figure(figsize=(12,8))
sns.boxplot(x='pop_size', y='mean', hue='approach', data=all_reconstructions, palette='Reds')
plt.legend(loc='best')

Plot relationship of performance for each DV over different approach parameterizations

In [ ]:
corr = approach_compare.corr(method='spearman')
mean_success = approach_compare.mean()
plot_df = approach_compare.join(retest_vals).join(avg_corr)
size = 2
f=sns.pairplot(plot_df.iloc[:,0:8], height=size,
             plot_kws={'color': [.4,.4,.4],
                       's': plot_df['avg_correlation']*1000},
             diag_kws={'bins': 20,
                      'edgecolor': 'k',
                      'linewidth': size/4})
axes = f.axes
# fix axes limits
for i in range(len(f.axes)):
    for j in range(len(f.axes)):
        ax = axes[i][j]
        ax.set_ylim([.2,1.1])
        ax.tick_params(left=False, bottom=False,
                      labelleft=False, labelbottom=False)
        if i!=j:
            ax.set_xlim([.2,1.1])
            ax.plot(ax.get_xlim(), ax.get_ylim(), lw=size, ls="--", c=".3", zorder=-1)
        if j<i:
            x = .6; y = .3
            if mean_success[j] > mean_success[i]:
                x = .28; y = 1
            ax.text(x, y, r'$\rho$ = %s' % format_num(corr.iloc[i,j]),
                   fontsize=size*8)
        # change sizing for upper triangle based on icc
        if j>i: 
            ax.set_visible(False)
            #ax.collections[0].set_sizes(plot_df['icc']**2*100)
            
# color diagonal
for i,ax in enumerate(f.diag_axes):
    ax.set_title(axes[i][0].get_ylabel(), color=colors[i%4], fontsize=size*10)
    for patch in ax.patches:
        patch.set_facecolor(colors[i%4])
        
# color labels
for i in range(len(f.axes)):
    left_ax = axes[i][0]
    bottom_ax = axes[-1][i]
    left_ax.set_ylabel(left_ax.get_ylabel(), color=colors[i%4],labelpad=10, fontsize=size*10)
    bottom_ax.set_xlabel('')
    
# set tick spacing
ax = axes[-1][-2]
ax.tick_params(length=1, width=1, labelleft=True, labelbottom=True)
ax.set_xticks([.18, 1])
ax.set_xticklabels(['0.2', '1.0'], fontsize=size*8, fontweight='bold')
ax.set_yticks([1])
ax.set_yticklabels(['1.0'], fontsize=size*8, fontweight='bold')
# common X
f.fig.text(0.5, 0.02, 'Average DV Reconstruction Score', ha='center', fontsize=size*10)
if save:
    save_figure(f, path.join(plot_dir, 'SFig1_cross_approach_correlations.png'), save_kws={'dpi': 300})

### K Nearest Visualization (Example)

#### Average Performance by Model Parameters

In [ ]:
plot_df = k_var_summary.reset_index()
sns.set_context('talk')
n_cols = 1
n_rows = len(pop_sizes)//n_cols
f, axes = plt.subplots(n_rows, n_cols, figsize=(12,n_rows*6))
axes = f.get_axes()
legend_on=True
for ax, pop_size in zip(axes, pop_sizes):
    sns.pointplot(x='k', y='corr_score', hue='weighting', 
                data=plot_df.query('pop_size==%s' % pop_size),
                ax=ax, dodge=.3, alpha=.75, join=False, ci=None)
    ax.set_title('Simulated Population Size: %s' % pop_size)
    ax.set_ylim(-1.1,1.1)
    ax.legend().set_visible(legend_on)
    legend_on=False
plt.subplots_adjust(hspace=.4)

#### Performance for each DV

Only taking the best parameters from the k-nearest neighbor algorithm

In [ ]:
var = "simon.hddm_drift"
ax = reconstructions['KNN'].query('var == "%s" and pop_size==100' % var).corr_score.hist(bins=30,
                                                                          edgecolor='white',
                                                                           figsize=[10,6])
ax.set_xlabel('Reconstruction Score', fontsize=40, labelpad=30)
ax.set_yticklabels([])
ax.set_yticks([])
ax.xaxis.set_major_locator(ticker.MultipleLocator(.05))
ax.tick_params(labelsize=30)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.grid(False)

##### Histogram of DV reconstruction scores

In [ ]:
plot_reconstruction_hist(reconstructions['KNN'], title='KNN Reconstruction', size=14)
plot_reconstruction_hist(reconstructions['RidgeCV'], title='RidgeCV Reconstruction', size=14)

In [ ]:
# saving
if save:
    plot_reconstruction_hist(reconstructions['KNN'], title='KNN Reconstruction', size=14,
                            filename=path.join(plot_dir, 'KNN_reconstruction.png'))
    plot_reconstruction_hist(reconstructions['RidgeCV'], title='RidgeCV Reconstruction', size=14,
                            filename=path.join(plot_dir, 'RidgeCV_reconstruction.png'))

There is clearly a bit of variability in the reconstruction accuracy based on the variable itself. While this variability narrows with larger populations, it's still there, and there are a few variables that cannot be reconstructed at all

One possibility is that the least reliable variables are the worst reconstructed. Let's look at that...

##### Reconstruction score vs. Reliability

In [ ]:
sns.set_context('talk')
f, axes = plt.subplots(1,2,figsize=(14,6))
for i, pop_size in enumerate(pop_sizes):
    sns.regplot('icc', 'mean', 
                data=reconstruction_summaries['KNN'].query('pop_size==%s' % pop_size), 
                label=pop_size, ax=axes[0], color=colors[i])
    sns.regplot('icc', 'std', 
                data=reconstruction_summaries['KNN'].query('pop_size==%s' % pop_size), 
                label=pop_size, ax=axes[1], color=colors[i])
axes[1].legend()
plt.suptitle('Reliability vs Reconstruction Score Mean/Std')
plt.subplots_adjust(wspace=.3)

We can dive in and look at one high/mediun/low reliable variable to see the reconstruction performance

In [ ]:
sorted_retest_vals = retest_vals.sort_values().index
N = len(sorted_retest_vals)
high_var = sorted_retest_vals[N-1]
med_var = sorted_retest_vals[N//2]
low_var = sorted_retest_vals[0]

In [ ]:
f, axes = plt.subplots(1,3, figsize=(20,8))
for ax, var in zip(axes, [high_var, med_var, low_var]):
    retest_in = var.replace('.logTr','').replace('.ReflogTr','')
    reliability = format_num(retest_data.loc[retest_in]['icc'])
    plot_df = k_reconstruction.query('var == "%s" and label=="partial_reconstruct"' % var)
    sns.boxplot(x='pop_size', y='corr_score', data=plot_df,  ax=ax)
    ax.set_title('%s\nICC: %s' % (var, reliability))
plt.subplots_adjust(wspace=.6)

So that doesn't seem to be the whole story. Instead, what might be important is the DVs actual relationship to the ontology. That is, some DVs are better captured by the ontology to begin with. Maybe DVs "peripheral" to this particular ontology (not well captured by the space) are poorly reconstructed.

We can look at this by looking at the relationship between communality and DV reconstruction

##### Reconstruction Score vs Communality

In [ ]:
sns.set_context('talk')
f, axes = plt.subplots(1,2,figsize=(14,6))
for i, pop_size in enumerate(pop_sizes):
    sns.regplot('communality', 'mean', 
                data=reconstruction_summaries['KNN'].query('pop_size==%s' % pop_size), 
                label=pop_size, ax=axes[0], color=colors[i])
    sns.regplot('communality', 'std', 
                data=reconstruction_summaries['KNN'].query('pop_size==%s' % pop_size), 
                label=pop_size, ax=axes[1], color=colors[i])
axes[1].legend()
plt.suptitle('Communality vs Reconstruction Score Mean/Std')
plt.subplots_adjust(wspace=.3)

It seems clear that DVs with poor communality are not reconstructed well. A less "analysis based" way to think about this is reconstruction will be worse if you are far away from the other variables in the set.

##### Reconstruction Score vs Average Correlation

In [ ]:
sns.set_context('talk')
f, axes = plt.subplots(1,2,figsize=(14,6))
for i, pop_size in enumerate(pop_sizes):
    sns.regplot('avg_correlation', 'mean', 
                data=reconstruction_summaries['KNN'].query('pop_size==%s' % pop_size), 
                label=pop_size, ax=axes[0], color=colors[i])
    sns.regplot('avg_correlation', 'std', 
                data=reconstruction_summaries['KNN'].query('pop_size==%s' % pop_size), 
                label=pop_size, ax=axes[1], color=colors[i])
axes[1].legend()
plt.suptitle('Average Correlation vs Reconstruction Score Mean/Std')
plt.subplots_adjust(wspace=.3)

It seems that the correlation with the overall dataset is important for reconstruction. All of this says that ontological mapping will be more successful if you have an a-priori reason to believe your new variable has something to do with the rest of the variables in the ontology. The weaker you believe that bond, the more data you should collect to articulate the connection

#### Visualization of classification success

In [ ]:
def plot_CV_predict(scores, pop_sizes, size):
    f, axes = plt.subplots(len(pop_sizes), 2, figsize=(size*2, len(pop_sizes)*size))
    if len(axes.shape) == 1:
        axes = [axes]
    for ax, pop_size in zip(axes, pop_sizes):
        precision, recall, f1, confusion = [scores[pop_size][i] for i in ['precision',
                                                                          'recall',
                                                                          'f1',
                                                                          'confusion']]
        # plot cross validated scores
        sns.heatmap(confusion, ax=ax[0], square=True, xticklabels=False, yticklabels=False, cbar=False)
        ax[0].set_title('Pop_size: %s, Recall: %s, Precision: %s, F1: %s' % (format_num(pop_size),
                                                                          format_num(precision), 
                                                                          format_num(recall), 
                                                                          format_num(f1)),
                    y=1.1)
        # plot labels of true values
        if 'true_confusion' in scores[pop_size].keys():
            sns.heatmap(scores[pop_size]['true_confusion'], ax=ax[1], square=True,
                        xticklabels=False, yticklabels=False)
            ax[1].set_title('Labels of true embedding')
        else:
            ax[1].set_visible(False)

In [ ]:
scores = classification_scores['RidgeCV']['var_scores']
keys = sorted(classification_scores['RidgeCV']['var_scores'].keys())
plot_CV_predict(scores,keys, 6)

In [ ]:
plot_CV_predict(cluster_scores, pop_sizes, 6)

In [ ]:
plot_CV_predict(noisy_scores, sorted(noisy_scores.keys()), 6)

In [ ]:
plot_distance_recon(reconstructed_distances, orig_distances, size=12)

In [ ]:
# save
if save:
    plot_distance_recon(reconstructed_distances, orig_distances, size=15, 
                       filename=path.join(plot_dir, 'distance_reconstructions.png'))

#### Visualization of Variability

##### Visualizing each factor's reconstruction separately

In [ ]:
plot_factor_reconstructions(reconstructions['KNN'], size=15, plot_diagonal=True, plot_regression=False)
plot_factor_reconstructions(reconstructions['RidgeCV'], size=15, plot_diagonal=True, plot_regression=False)

In [ ]:
# save
if save:
    plot_factor_reconstructions(reconstructions['KNN'], size=10, plot_diagonal=True, plot_regression=False,
                                filename=path.join(plot_dir, 'KNN_factor_reconstructions.png'))
    plot_factor_reconstructions(reconstructions['RidgeCV'], size=10, plot_diagonal=True, plot_regression=False,
                                filename=path.join(plot_dir, 'RidgeCV_factor_reconstructions.png'))

##### Using TSNE

More complicate, we can visualize this by looking at the MDS plotting:
1. The original DVs
2. The "best" reconstruction using all the data
3. The n_reps simulated estimates with a smaller population size

In [ ]:
plot_reconstruction_2D(k_reconstruction, n_reps=30, n_colored=6, use_background=True)

### Save Visualizations

In [ ]:
if save:
    for name, reconstruction in reconstructions.items():
        plot_reconstruction_hist(reconstruction, title='KNN Reconstruction', size=14,
                                filename=path.join(plot_dir, name+'_recon_hist.pdf'))
        plot_factor_reconstructions(reconstruction, size=10,
                                   filename=path.join(plot_dir, name+'_factor_recon.pdf'))

## Reduced Reconstruction using fewer contextualizing variables